In [1]:
import ccxt
import pandas as pd
import numpy as np
import os
import zipfile
import time
import logging
import requests
import io
from datetime import date, datetime, timezone, timedelta
from tqdm import tqdm
from config import DATA_LOCATION, PROXIES,START_DATE,END_DATE
from utils import format_symbol, get_ms_from_midnight

logger = logging.getLogger(__name__)
# datetime set time zone to UTC
os.environ['TZ'] = 'UTC'
time.tzset()

In [ ]:
def fetch_depth_range_cryptofuture(symbol, since_ms, until_ms, margin_type="um", data_source="binancevision",
                                  )-> list:
    """fetch order book snapshots for crypto futures from [BinanceVision](https://data.binance.vision/?prefix=data/futures/margin_type/daily/bookDepth/)
    
    :note: data is 60s interval snapshots by default.
    
    :return: (list)
    
    """

    if data_source != "binancevision":
        raise NotImplementedError("Only binancevision data source is supported for depth data.")
    # if step_ms not in [0, 60000]:
    #     raise ValueError("Only step_ms of 0 (no alignment) or 60000 (1 minute) is supported to ensure data quality.")
    # if not fill_start_of_day:
    #     raise NotImplementedError("Currently only fill_start_of_day=True is supported to ensure data continuity at midnight.")

    # snapshot_freq_ms=30*1000
    # if data_source == "binancevision":
    #     snapshot_freq_ms=30*1000 # 30s per snapshot
        
    all_data = []
    # if fill_start_of_day:
    #     current_since = since_ms - 24*60*60*1000 # move to the previous day to fill the 0:00 snapshot of since_ms
    # else:
    #     current_since = since_ms
    # last_snapshots = []
    current_since = since_ms


    # Track processed dates to avoid re-downloading same daily file
    processed_dates = set()
        
    pbar = tqdm(total=until_ms - since_ms, desc=f"Fetching Depth {symbol}")
    # df=None

    while current_since < until_ms:
        dt = datetime.fromtimestamp(current_since / 1000, tz=timezone.utc)
        date_str = dt.strftime("%Y-%m-%d")
        
        if date_str in processed_dates:
            # Advance to next day start to avoid infinite loop
            next_day = (dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1))
            current_since = int(next_day.timestamp() * 1000)
            continue
            
        formatted_symbol = format_symbol(symbol).upper()
        # Correct URL pattern: includes symbol subdirectory
        url = f"https://data.binance.vision/data/futures/{margin_type}/daily/bookDepth/{formatted_symbol}/{formatted_symbol}-bookDepth-{date_str}.zip"
        
        try:
            response = requests.get(url, proxies=PROXIES, timeout=30)
            if response.status_code == 200:
                with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
                    csv_name = zf.namelist()[0]
                    with zf.open(csv_name) as f:
                        # Use low_memory=False to avoid DtypeWarning
                        df = pd.read_csv(f, low_memory=False)

                        # Sample data columns: timestamp,percentage,depth,notional
                        if 'timestamp' in df.columns:
                            df['dt'] = pd.to_datetime(df['timestamp'])
                            if df['dt'].dt.tz is None:
                                df['dt'] = df['dt'].dt.tz_localize('UTC')
                            
                            # Convert to milliseconds since epoch robustly
                            epoch = pd.Timestamp("1970-01-01", tz='UTC')
                            df['ms_original'] = (df['dt'] - epoch) // pd.Timedelta(milliseconds=1)
                        

                            # 1. Align to the nearest snapshot_freq boundary
                            # Use round to ensure snapshots close to the boundary are mapped correctly.
                            # Exact matches (e.g., 00s, 30s) will be mapped to themselves.
                            df['ms_aligned'] = (df['ms_original'] / snapshot_freq_ms).round().astype(np.int64) * snapshot_freq_ms

                            # 2. Filter out "30s" snapshots if we only want minute data (step_ms=60000)
                            if step_ms > 0:
                                df = df[df['ms_aligned'] % step_ms == 0]

                            if df.empty:
                                continue

                            # 3. Handle multiple snapshots mapping to the same ms_aligned
                            # Rule: Prefer exact match (ms_original == ms_aligned), otherwise latest ms_original
                            df['is_exact'] = (df['ms_original'] == df['ms_aligned'])
                            # Sort by ms_aligned, then priority (exact match > latest)
                            df_sorted = df.sort_values(['ms_aligned', 'is_exact', 'ms_original'], 
                                                       ascending=[True, False, False])
                            # Pick the best ms_original for each ms_aligned
                            best_originals = df_sorted.drop_duplicates('ms_aligned')[['ms_aligned', 'ms_original']]
                            # Merge back to keep only rows from the best snapshots
                            df_best = df.merge(best_originals, on=['ms_aligned', 'ms_original'])

                            # 4. Construct groundtruth index (ms_aligned and percentage) and merge
                            # This ensures timestamps are exactly on the grid and consistent.
                            mask = (df_best['ms_aligned'] >= since_ms) & (df_best['ms_aligned'] < until_ms)
                            df_in_range = df_best.loc[mask]
                            
                            if not df_in_range.empty:
                                day_ms_aligned = sorted(df_in_range['ms_aligned'].unique())
                                day_percentages = sorted(df_in_range['percentage'].unique())
                                
                                groundtruth = pd.MultiIndex.from_product(
                                    [day_ms_aligned, day_percentages], 
                                    names=['ms_aligned', 'percentage']
                                ).to_frame(index=False)
                                
                                # Merge data into groundtruth to anchor everything to the aligned grid
                                df_filtered = pd.merge(groundtruth, df_in_range, on=['ms_aligned', 'percentage'], how='left')
                                
                                # Drop NaNs (where a snapshot didn't have a particular percentage level)
                                # and final safety deduplication
                                df_filtered = df_filtered.dropna(subset=['ms_original']).drop_duplicates(['ms_aligned', 'percentage'])
                                
                                # Add ms_midnight column (milliseconds since midnight UTC)
                                df_filtered['dt_aligned'] = pd.to_datetime(df_filtered['ms_aligned'], unit='ms', utc=True)
                                df_filtered['ms_midnight'] = df_filtered['dt_aligned'].apply(get_ms_from_midnight)
                                
                                # Convert to list of dicts for return
                                records = df_filtered.to_dict('records')
                                
                                # If we have data from previous day, fill the beginning of the current day
                                # to ensure continuity if there's a gap at midnight.
                                day_start_ms = int(dt.replace(hour=0, minute=0, second=0, microsecond=0).timestamp() * 1000)
                                first_ms_in_day = df_filtered['ms_aligned'].min()
                                
                                if last_snapshots and first_ms_in_day > day_start_ms:
                                    fill_records = []
                                    for snap in last_snapshots:
                                        fill_snap = snap.copy()
                                        fill_snap['ms_aligned'] = day_start_ms
                                        fill_records.append(fill_snap)
                                    all_data.extend(fill_records)

                                all_data.extend(records)
                                
                                # Update last_snapshots for the next day's gap filling
                                max_ms = df_filtered['ms_aligned'].max()
                                last_snapshots = [r for r in records if r['ms_aligned'] == max_ms]
            elif response.status_code == 404:
                logger.debug(f"No depth data for {symbol} on {date_str} (404)")
            else:
                logger.warning(f"Failed to download depth data for {symbol} on {date_str}: {response.status_code}")
        except Exception as e:
            logger.error(f"Error fetching depth data for {symbol} on {date_str}: {e}")
            
        processed_dates.add(date_str)
        # Move current_since to next day start
        next_day = (dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1))
        new_since = int(next_day.timestamp() * 1000)
        
        # Update progress bar
        progress = new_since - current_since
        if progress > 0:
            pbar.update(min(progress, until_ms - current_since))
        
        current_since = new_since

    pbar.close()
    return all_data


In [55]:
symbol="BTCUSDT"
since_ms=datetime.fromisoformat(START_DATE).timestamp()*1000
until_ms=(datetime.fromisoformat(START_DATE)+timedelta(days=1)).timestamp()*1000
step_ms=60000
margin_type="um"
data_source="binancevision"
fill_start_of_day=True


In [86]:

if data_source != "binancevision":
    raise NotImplementedError("Only binancevision data source is supported for depth data.")
if step_ms not in [0, 60000]:
    raise ValueError("Only step_ms of 0 (no alignment) or 60000 (1 minute) is supported to ensure data quality.")
if not fill_start_of_day:
    raise NotImplementedError("Currently only fill_start_of_day=True is supported to ensure data continuity at midnight.")
assert since_ms%24*60*60*1000==0, "since_ms must be aligned to the start of a day (00:00 UTC) to ensure proper snapshot alignment and filling."

snapshot_freq_ms=30*1000
if data_source == "binancevision":
    snapshot_freq_ms=30*1000 # 30s per snapshot

# use snapshot_freq_ms to construct base index. This ensures we have a complete grid of expected timestamps and percentage levels to merge with, which is crucial for handling missing data and ensuring consistency.
# some times we only have a <30s snapshot with out a >30s, so if we use step_ms=60000 to construct the index, we will miss those snapshots and have no way to merge them back in, resulting in data loss and inconsistency.
res_df=pd.MultiIndex.from_product(
    [[int(since_ms + i*snapshot_freq_ms) for i in range(int((until_ms - since_ms) // snapshot_freq_ms))], 
     [-5,-4,-3,-2,-1,1,2,3,4,5]], 
    names=['ms_aligned', 'percentage']
).to_frame(index=False)
res_df
    

,ms_aligned,percentage
0,1767225600000,-5
1,1767225600000,-4
2,1767225600000,-3
3,1767225600000,-2
4,1767225600000,-1
...,...,...
28795,1767311970000,1
28796,1767311970000,2
28797,1767311970000,3
28798,1767311970000,4


In [88]:

all_data = []
if fill_start_of_day:
    current_since = since_ms - 24*60*60*1000 # move to the previous day to fill the 0:00 snapshot of since_ms
else:
    current_since = since_ms
last_snapshots = []

# Track processed dates to avoid re-downloading same daily file
processed_dates = set()
    
pbar = tqdm(total=until_ms - since_ms, desc=f"Fetching Depth {symbol}")
df=None

while current_since < until_ms:
    dt = datetime.fromtimestamp(current_since / 1000, tz=timezone.utc)
    date_str = dt.strftime("%Y-%m-%d")
    
    if date_str in processed_dates:
        # Advance to next day start to avoid infinite loop
        next_day = (dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1))
        current_since = int(next_day.timestamp() * 1000)
        continue
        
    formatted_symbol = format_symbol(symbol).upper()
    # Correct URL pattern: includes symbol subdirectory
    url = f"https://data.binance.vision/data/futures/{margin_type}/daily/bookDepth/{formatted_symbol}/{formatted_symbol}-bookDepth-{date_str}.zip"
    
    try:
        response = requests.get(url, proxies=PROXIES, timeout=30)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
                csv_name = zf.namelist()[0]
                with zf.open(csv_name) as f:
                    # Use low_memory=False to avoid DtypeWarning
                    df = pd.read_csv(f, low_memory=False)
                    origin_df=df.copy()

                    # # Sample data columns: timestamp,percentage,depth,notional
                    # if 'timestamp' in df.columns:
                    #     df['dt'] = pd.to_datetime(df['timestamp'])
                    #     if df['dt'].dt.tz is None:
                    #         df['dt'] = df['dt'].dt.tz_localize('UTC')
                        
                    #     # Convert to milliseconds since epoch robustly
                    #     epoch = pd.Timestamp("1970-01-01", tz='UTC')
                    #     df['ms_original'] = (df['dt'] - epoch) // pd.Timedelta(milliseconds=1)
                    

                    #     # 1. Align to the nearest next snapshot_freq boundary
                    #     # Use round to ensure snapshots close to the boundary are mapped correctly.
                    #     # Exact matches (e.g., 00s, 30s) will be mapped to themselves.
                    #     df['ms_aligned'] = (df['ms_original'] / snapshot_freq_ms).round().astype(np.int64) * snapshot_freq_ms

                    #     # 2. Filter out "30s" snapshots if we only want minute data (step_ms=60000)
                    #     if step_ms > 0:
                    #         df = df[df['ms_aligned'] % step_ms == 0]

                    #     if df.empty:
                    #         continue

                    #     # 3. Handle multiple snapshots mapping to the same ms_aligned
                    #     # Rule: Prefer exact match (ms_original == ms_aligned), otherwise latest ms_original
                    #     df['is_exact'] = (df['ms_original'] == df['ms_aligned'])
                    #     # Sort by ms_aligned, then priority (exact match > latest)
                    #     df_sorted = df.sort_values(['ms_aligned', 'is_exact', 'ms_original'], 
                    #                                 ascending=[True, False, False])
                    #     # Pick the best ms_original for each ms_aligned
                    #     best_originals = df_sorted.drop_duplicates('ms_aligned')[['ms_aligned', 'ms_original']]
                    #     # Merge back to keep only rows from the best snapshots
                    #     df_best = df.merge(best_originals, on=['ms_aligned', 'ms_original'])

                    #     # 4. Construct groundtruth index (ms_aligned and percentage) and merge
                    #     # This ensures timestamps are exactly on the grid and consistent.
                    #     mask = (df_best['ms_aligned'] >= since_ms) & (df_best['ms_aligned'] < until_ms)
                    #     df_in_range = df_best.loc[mask]
                        
                    #     if not df_in_range.empty:
                    #         day_ms_aligned = sorted(df_in_range['ms_aligned'].unique())
                    #         day_percentages = sorted(df_in_range['percentage'].unique())
                            
                    #         groundtruth = pd.MultiIndex.from_product(
                    #             [day_ms_aligned, day_percentages], 
                    #             names=['ms_aligned', 'percentage']
                    #         ).to_frame(index=False)
                            
                    #         # Merge data into groundtruth to anchor everything to the aligned grid
                    #         df_filtered = pd.merge(groundtruth, df_in_range, on=['ms_aligned', 'percentage'], how='left')
                            
                    #         # Drop NaNs (where a snapshot didn't have a particular percentage level)
                    #         # and final safety deduplication
                    #         df_filtered = df_filtered.dropna(subset=['ms_original']).drop_duplicates(['ms_aligned', 'percentage'])
                            
                    #         # Add ms_midnight column (milliseconds since midnight UTC)
                    #         df_filtered['dt_aligned'] = pd.to_datetime(df_filtered['ms_aligned'], unit='ms', utc=True)
                    #         df_filtered['ms_midnight'] = df_filtered['dt_aligned'].apply(get_ms_from_midnight)
                            
                    #         # Convert to list of dicts for return
                    #         records = df_filtered.to_dict('records')
                            
                    #         # If we have data from previous day, fill the beginning of the current day
                    #         # to ensure continuity if there's a gap at midnight.
                    #         day_start_ms = int(dt.replace(hour=0, minute=0, second=0, microsecond=0).timestamp() * 1000)
                    #         first_ms_in_day = df_filtered['ms_aligned'].min()
                            
                    #         if last_snapshots and first_ms_in_day > day_start_ms:
                    #             fill_records = []
                    #             for snap in last_snapshots:
                    #                 fill_snap = snap.copy()
                    #                 fill_snap['ms_aligned'] = day_start_ms
                    #                 fill_records.append(fill_snap)
                    #             all_data.extend(fill_records)

                    #         all_data.extend(records)
                            
                    #         # Update last_snapshots for the next day's gap filling
                    #         max_ms = df_filtered['ms_aligned'].max()
                    #         last_snapshots = [r for r in records if r['ms_aligned'] == max_ms]
        elif response.status_code == 404:
            logger.debug(f"No depth data for {symbol} on {date_str} (404)")
        else:
            logger.warning(f"Failed to download depth data for {symbol} on {date_str}: {response.status_code}")
    except Exception as e:
        logger.error(f"Error fetching depth data for {symbol} on {date_str}: {e}")
        
    processed_dates.add(date_str)
    # Move current_since to next day start
    next_day = (dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1))
    new_since = int(next_day.timestamp() * 1000)
    
    # Update progress bar
    progress = new_since - current_since
    if progress > 0:
        pbar.update(min(progress, until_ms - current_since))
    
    current_since = new_since

pbar.close()

Fetching Depth BTCUSDT: 172800000.0it [00:01, 151359499.66it/s]                               


In [89]:
origin_df

,timestamp,percentage,depth,notional
0,2026-01-01 00:00:09,-5,6419.548,5.524643e+08
1,2026-01-01 00:00:09,-4,5971.786,5.149844e+08
2,2026-01-01 00:00:09,-3,5193.131,4.491309e+08
3,2026-01-01 00:00:09,-2,4120.377,3.576510e+08
4,2026-01-01 00:00:09,-1,2320.040,2.022281e+08
...,...,...,...,...
26865,2026-01-01 23:59:30,1,2418.263,2.155681e+08
26866,2026-01-01 23:59:30,2,5768.960,5.172847e+08
26867,2026-01-01 23:59:30,3,6301.089,5.656928e+08
26868,2026-01-01 23:59:30,4,7034.562,6.331192e+08


In [95]:
df = origin_df.copy()
# if df.empty:
#     continue
# Sample data columns: timestamp,percentage,depth,notional
if 'timestamp' in df.columns:
    df['dt'] = pd.to_datetime(df['timestamp'])
    if df['dt'].dt.tz is None:
        df['dt'] = df['dt'].dt.tz_localize('UTC')
    
    # Convert to milliseconds since epoch robustly
    epoch = pd.Timestamp("1970-01-01", tz='UTC')
    df['ms_original'] = (df['dt'] - epoch) // pd.Timedelta(milliseconds=1)


    # 1. Align to the nearest next snapshot_freq boundary
    # Use round to ensure snapshots close to the boundary are mapped correctly.
    # 00s, 30s will be mapped to next snapshot.
    df['ms_aligned'] = ((df['ms_original'] // snapshot_freq_ms)+1) * snapshot_freq_ms
    
    # ffill missing values for each percentage level, to ensure we have a complete grid of snapshots for each percentage level, which is crucial for training and evaluation consistency.
    # df = df.groupby('percentage',as_index=False).ffill()
    
    # drop duplicates keep last
    # res_df=res_df.merge(df.drop_duplicates(['ms_aligned','percentage'],keep='last'),on=['ms_aligned','percentage'],how='left')
        


In [98]:
df.groupby('percentage',as_index=False,group_keys=False).ffill()

,timestamp,depth,notional,dt,ms_original,ms_aligned
0,2026-01-01 00:00:09,6419.548,5.524643e+08,2026-01-01 00:00:09+00:00,1767225609000,1767225630000
1,2026-01-01 00:00:09,5971.786,5.149844e+08,2026-01-01 00:00:09+00:00,1767225609000,1767225630000
2,2026-01-01 00:00:09,5193.131,4.491309e+08,2026-01-01 00:00:09+00:00,1767225609000,1767225630000
3,2026-01-01 00:00:09,4120.377,3.576510e+08,2026-01-01 00:00:09+00:00,1767225609000,1767225630000
4,2026-01-01 00:00:09,2320.040,2.022281e+08,2026-01-01 00:00:09+00:00,1767225609000,1767225630000
...,...,...,...,...,...,...
26865,2026-01-01 23:59:30,2418.263,2.155681e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000
26866,2026-01-01 23:59:30,5768.960,5.172847e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000
26867,2026-01-01 23:59:30,6301.089,5.656928e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000
26868,2026-01-01 23:59:30,7034.562,6.331192e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000


In [76]:
df

,timestamp,percentage,depth,notional,dt,ms_original,ms_aligned
10,2026-01-01 00:00:58,-5,6498.737,5.593286e+08,2026-01-01 00:00:58+00:00,1767225658000,1767225660000
11,2026-01-01 00:00:58,-4,6042.729,5.211561e+08,2026-01-01 00:00:58+00:00,1767225658000,1767225660000
12,2026-01-01 00:00:58,-3,5269.363,4.557452e+08,2026-01-01 00:00:58+00:00,1767225658000,1767225660000
13,2026-01-01 00:00:58,-2,4197.055,3.643013e+08,2026-01-01 00:00:58+00:00,1767225658000,1767225660000
14,2026-01-01 00:00:58,-1,2392.803,2.085378e+08,2026-01-01 00:00:58+00:00,1767225658000,1767225660000
...,...,...,...,...,...,...,...
26865,2026-01-01 23:59:30,1,2418.263,2.155681e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000
26866,2026-01-01 23:59:30,2,5768.960,5.172847e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000
26867,2026-01-01 23:59:30,3,6301.089,5.656928e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000
26868,2026-01-01 23:59:30,4,7034.562,6.331192e+08,2026-01-01 23:59:30+00:00,1767311970000,1767312000000


In [ ]:
tmp_df=res_df.merge(df[df['percentage']==1],on=['ms_aligned','percentage'],how='left')
tmp_df=tmp_df[tmp_df['percentage']==1]
tmp_df

,ms_aligned,percentage,timestamp,depth,notional,dt,ms_original
5,1767225600000,1,NaN,NaN,NaN,NaT,NaN
15,1767225630000,1,2026-01-01 00:00:09,1127.950,9.914092e+07,2026-01-01 00:00:09+00:00,1.767226e+12
25,1767225660000,1,2026-01-01 00:00:58,1314.910,1.155787e+08,2026-01-01 00:00:58+00:00,1.767226e+12
35,1767225690000,1,2026-01-01 00:01:16,1317.149,1.157746e+08,2026-01-01 00:01:16+00:00,1.767226e+12
45,1767225720000,1,2026-01-01 00:01:46,1320.600,1.161620e+08,2026-01-01 00:01:46+00:00,1.767226e+12
...,...,...,...,...,...,...,...
28755,1767311850000,1,2026-01-01 23:57:01,2444.842,2.179487e+08,2026-01-01 23:57:01+00:00,1.767312e+12
28765,1767311880000,1,2026-01-01 23:57:31,2449.683,2.183689e+08,2026-01-01 23:57:31+00:00,1.767312e+12
28775,1767311910000,1,2026-01-01 23:58:01,2409.702,2.148350e+08,2026-01-01 23:58:01+00:00,1.767312e+12
28785,1767311940000,1,2026-01-01 23:58:32,2429.444,2.165854e+08,2026-01-01 23:58:32+00:00,1.767312e+12


In [ ]:



    # 3. Handle multiple snapshots mapping to the same ms_aligned
    # # Sort by ms_aligned, then priority (exact match > latest)
    # df_sorted = df.sort_values(['ms_aligned', 'ms_original'], 
    #                             ascending=[True, False])
    # Pick the best ms_original for each ms_aligned
    best_originals = df.drop_duplicates(['ms_aligned','percentage'],keep='last')[['ms_aligned', 'ms_original']]
    # Merge back to keep only rows from the best snapshots
    df_best = df.merge(best_originals, on=['ms_aligned', 'ms_original'])

    # 4. Construct groundtruth index (ms_aligned and percentage) and merge
    # This ensures timestamps are exactly on the grid and consistent.
    mask = (df_best['ms_aligned'] >= since_ms) & (df_best['ms_aligned'] < until_ms)
    df_in_range = df_best.loc[mask]
    
    if not df_in_range.empty:
        day_ms_aligned = sorted(df_in_range['ms_aligned'].unique())
        day_percentages = sorted(df_in_range['percentage'].unique())
        
        groundtruth = pd.MultiIndex.from_product(
            [day_ms_aligned, day_percentages], 
            names=['ms_aligned', 'percentage']
        ).to_frame(index=False)
        
        # Merge data into groundtruth to anchor everything to the aligned grid
        df_filtered = pd.merge(groundtruth, df_in_range, on=['ms_aligned', 'percentage'], how='left')
        
        # Drop NaNs (where a snapshot didn't have a particular percentage level)
        # and final safety deduplication
        df_filtered = df_filtered.dropna(subset=['ms_original']).drop_duplicates(['ms_aligned', 'percentage'])
        
        # Add ms_midnight column (milliseconds since midnight UTC)
        df_filtered['dt_aligned'] = pd.to_datetime(df_filtered['ms_aligned'], unit='ms', utc=True)
        df_filtered['ms_midnight'] = df_filtered['dt_aligned'].apply(get_ms_from_midnight)
        
        # last step. Filter out "30s" snapshots if we only want minute data (step_ms=60000). this step must be in the end, to make sure we can fill the missing 30s snapshots back in if we only want minute data, to avoid data loss and inconsistency.
        if step_ms > 0:
            df_filtered = df_filtered[df_filtered['ms_aligned'] % step_ms == 0]
        
        # Convert to list of dicts for return
        records = df_filtered.to_dict('records')
        
        # If we have data from previous day, fill the beginning of the current day
        # to ensure continuity if there's a gap at midnight.
        day_start_ms = int(dt.replace(hour=0, minute=0, second=0, microsecond=0).timestamp() * 1000)
        first_ms_in_day = df_filtered['ms_aligned'].min()
        
        if last_snapshots and first_ms_in_day > day_start_ms:
            fill_records = []
            for snap in last_snapshots:
                fill_snap = snap.copy()
                fill_snap['ms_aligned'] = day_start_ms
                fill_records.append(fill_snap)
            all_data.extend(fill_records)

        all_data.extend(records)
        
        # Update last_snapshots for the next day's gap filling
        max_ms = df_filtered['ms_aligned'].max()
        last_snapshots = [r for r in records if r['ms_aligned'] == max_ms]

In [54]:
df

,timestamp,percentage,depth,notional,dt,ms_original,ms_aligned
20,2026-01-02 00:01:16,-5,6030.189,5.243137e+08,2026-01-02 00:01:16+00:00,1767312076000,1767312120000
21,2026-01-02 00:01:16,-4,5174.523,4.517868e+08,2026-01-02 00:01:16+00:00,1767312076000,1767312120000
22,2026-01-02 00:01:16,-3,4446.625,3.893772e+08,2026-01-02 00:01:16+00:00,1767312076000,1767312120000
23,2026-01-02 00:01:16,-2,3072.834,2.702749e+08,2026-01-02 00:01:16+00:00,1767312076000,1767312120000
24,2026-01-02 00:01:16,-1,1613.371,1.426534e+08,2026-01-02 00:01:16+00:00,1767312076000,1767312120000
...,...,...,...,...,...,...,...
27225,2026-01-02 23:59:31,1,1634.510,1.476713e+08,2026-01-02 23:59:31+00:00,1767398371000,1767398400000
27226,2026-01-02 23:59:31,2,2996.034,2.719585e+08,2026-01-02 23:59:31+00:00,1767398371000,1767398400000
27227,2026-01-02 23:59:31,3,3902.723,3.554995e+08,2026-01-02 23:59:31+00:00,1767398371000,1767398400000
27228,2026-01-02 23:59:31,4,4315.175,3.938851e+08,2026-01-02 23:59:31+00:00,1767398371000,1767398400000


In [66]:
df=pd.DataFrame.from_records(all_data,index=None)
df

,ms_aligned,percentage,timestamp,depth,notional,dt,ms_original,dt_aligned,ms_midnight
0,1767225720000,-5,2026-01-01 00:01:16,6525.423,5.616601e+08,2026-01-01 00:01:16+00:00,1767225676000,2026-01-01 00:02:00+00:00,120000
1,1767225720000,-4,2026-01-01 00:01:16,6069.343,5.234815e+08,2026-01-01 00:01:16+00:00,1767225676000,2026-01-01 00:02:00+00:00,120000
2,1767225720000,-3,2026-01-01 00:01:16,5295.904,4.580644e+08,2026-01-01 00:01:16+00:00,1767225676000,2026-01-01 00:02:00+00:00,120000
3,1767225720000,-2,2026-01-01 00:01:16,4223.554,3.666170e+08,2026-01-01 00:01:16+00:00,1767225676000,2026-01-01 00:02:00+00:00,120000
4,1767225720000,-1,2026-01-01 00:01:16,2412.929,2.102999e+08,2026-01-01 00:01:16+00:00,1767225676000,2026-01-01 00:02:00+00:00,120000
...,...,...,...,...,...,...,...,...,...
12965,1767311940000,1,2026-01-01 23:58:32,2429.444,2.165854e+08,2026-01-01 23:58:32+00:00,1767311912000,2026-01-01 23:59:00+00:00,86340000
12966,1767311940000,2,2026-01-01 23:58:32,5775.488,5.178898e+08,2026-01-01 23:58:32+00:00,1767311912000,2026-01-01 23:59:00+00:00,86340000
12967,1767311940000,3,2026-01-01 23:58:32,6307.727,5.663103e+08,2026-01-01 23:58:32+00:00,1767311912000,2026-01-01 23:59:00+00:00,86340000
12968,1767311940000,4,2026-01-01 23:58:32,7041.554,6.337718e+08,2026-01-01 23:58:32+00:00,1767311912000,2026-01-01 23:59:00+00:00,86340000


In [33]:
df = fetch_depth_range_cryptofuture(symbol, since_ms, until_ms, step_ms, margin_type, data_source)

Fetching Depth BTCUSDT: 259200000.0it [00:45, 5681879.71it/s]                                  


In [34]:
df=pd.DataFrame.from_records(df,index=None)
df

,ms_aligned,percentage,timestamp,depth,notional,dt,ms_original,is_exact,dt_aligned,ms_midnight
0,1767225600000,-5,2026-01-01 00:00:09,6419.548,5.524643e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
1,1767225600000,-4,2026-01-01 00:00:09,5971.786,5.149844e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
2,1767225600000,-3,2026-01-01 00:00:09,5193.131,4.491309e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
3,1767225600000,-2,2026-01-01 00:00:09,4120.377,3.576510e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
4,1767225600000,-1,2026-01-01 00:00:09,2320.040,2.022281e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
...,...,...,...,...,...,...,...,...,...,...
26895,1767398340000,1,2026-01-02 23:59:01,1710.788,1.545749e+08,2026-01-02 23:59:01+00:00,1767398341000,False,2026-01-02 23:59:00+00:00,86340000
26896,1767398340000,2,2026-01-02 23:59:01,3081.166,2.796609e+08,2026-01-02 23:59:01+00:00,1767398341000,False,2026-01-02 23:59:00+00:00,86340000
26897,1767398340000,3,2026-01-02 23:59:01,3991.695,3.635519e+08,2026-01-02 23:59:01+00:00,1767398341000,False,2026-01-02 23:59:00+00:00,86340000
26898,1767398340000,4,2026-01-02 23:59:01,4400.348,4.015800e+08,2026-01-02 23:59:01+00:00,1767398341000,False,2026-01-02 23:59:00+00:00,86340000


In [35]:
df['ms_midnight'].diff().value_counts()

ms_midnight
 0.0           24210
 60000.0        2566
 120000.0         72
 180000.0         32
 240000.0         18
-86340000.0        1
Name: count, dtype: int64

In [36]:
df[df['ms_midnight']==0]

,ms_aligned,percentage,timestamp,depth,notional,dt,ms_original,is_exact,dt_aligned,ms_midnight
0,1767225600000,-5,2026-01-01 00:00:09,6419.548,5.524643e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
1,1767225600000,-4,2026-01-01 00:00:09,5971.786,5.149844e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
2,1767225600000,-3,2026-01-01 00:00:09,5193.131,4.491309e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
3,1767225600000,-2,2026-01-01 00:00:09,4120.377,3.576510e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
4,1767225600000,-1,2026-01-01 00:00:09,2320.040,2.022281e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
5,1767225600000,1,2026-01-01 00:00:09,1127.950,9.914092e+07,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
6,1767225600000,2,2026-01-01 00:00:09,2440.403,2.158238e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
7,1767225600000,3,2026-01-01 00:00:09,3337.197,2.963951e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
8,1767225600000,4,2026-01-01 00:00:09,3987.790,3.553787e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0
9,1767225600000,5,2026-01-01 00:00:09,5514.242,4.947888e+08,2026-01-01 00:00:09+00:00,1767225609000,False,2026-01-01 00:00:00+00:00,0


In [25]:
margin_type='um'
formatted_symbol='BTCUSDT'
date_str = '2025-12-31'
url = f"https://data.binance.vision/data/futures/{margin_type}/daily/bookDepth/{formatted_symbol}/{formatted_symbol}-bookDepth-{date_str}.zip"
response = requests.get(url, proxies=PROXIES, timeout=30)
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
        csv_name = zf.namelist()[0]
        with zf.open(csv_name) as f:
            # Use low_memory=False to avoid DtypeWarning
            df = pd.read_csv(f, low_memory=False)

In [30]:
df.tail(20)

,timestamp,percentage,depth,notional
27170,2025-12-31 23:59:00,-5,6645.485,5.721776e+08
27171,2025-12-31 23:59:00,-4,6197.671,5.346933e+08
27172,2025-12-31 23:59:00,-3,5418.652,4.688090e+08
27173,2025-12-31 23:59:00,-2,4344.683,3.772256e+08
27174,2025-12-31 23:59:00,-1,2530.856,2.206323e+08
27175,2025-12-31 23:59:00,1,1282.215,1.127168e+08
27176,2025-12-31 23:59:00,2,2604.431,2.302638e+08
27177,2025-12-31 23:59:00,3,3501.366,3.108472e+08
27178,2025-12-31 23:59:00,4,4152.691,3.698967e+08
27179,2025-12-31 23:59:00,5,5679.275,5.093188e+08


In [113]:
def fetch_depth_range_cryptofuture(symbol, since_ms, until_ms, margin_type="um", data_source="binancevision"):
    """fetch order book snapshots for crypto futures from [BinanceVision](https://data.binance.vision/?prefix=data/futures/margin_type/daily/bookDepth/)
    depth data has no resolution parameter, and is always about 30s interval snapshots. you should align the timestamps to the nearest next minute boundary (e.g., 60s) to ensure consistency, and fill missing snapshots at the start of the day if needed to ensure continuity at midnight.
    
    :return: (pd.DataFrame)
    """

    if data_source != "binancevision":
        raise NotImplementedError("Only binancevision data source is supported for depth data.")

    all_dfs_dict = {}
    processed_dates = set()
    current_since = since_ms - 24*60*60*1000 # start from the previous day to fill the 0:00 snapshot of since_ms
        
    pbar = tqdm(total=until_ms - since_ms, desc=f"Fetching Depth {symbol}")

    while current_since < until_ms:
        dt = datetime.fromtimestamp(current_since / 1000, tz=timezone.utc)
        date_str = dt.strftime("%Y-%m-%d")
        date_str_yyyymmdd = dt.strftime("%Y%m%d")
        
        if date_str in processed_dates:
            next_day = (dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1))
            current_since = int(next_day.timestamp() * 1000)
            continue
            
        formatted_symbol = format_symbol(symbol).upper()
        url = f"https://data.binance.vision/data/futures/{margin_type}/daily/bookDepth/{formatted_symbol}/{formatted_symbol}-bookDepth-{date_str}.zip"
        
        try:
            response = requests.get(url, proxies=PROXIES, timeout=30)
            if response.status_code == 200:
                with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
                    csv_name = zf.namelist()[0]
                    with zf.open(csv_name) as f:
                        df = pd.read_csv(f, low_memory=False)
                        all_dfs_dict[date_str_yyyymmdd] = df
                        
            elif response.status_code == 404:
                logger.debug(f"No depth data for {symbol} on {date_str} (404)")
            else:
                logger.warning(f"Failed to download depth data for {symbol} on {date_str}: {response.status_code}")
        except Exception as e:
            logger.error(f"Error fetching depth data for {symbol} on {date_str}: {e}")
            
        processed_dates.add(date_str)
        next_day = (dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(days=1))
        new_since = int(next_day.timestamp() * 1000)
        
        progress = new_since - current_since
        if progress > 0:
            pbar.update(min(progress, until_ms - current_since))
        current_since = new_since

    pbar.close()
    if not all_dfs_dict:
        return {}
    return all_dfs_dict


def save_depth_data(symbol,date_str:str, depth_data:pd.DataFrame, asset_class="cryptofuture"):
    """
    Save depth data to LEAN format.
    Format: ms_midnight, percentage, depth, notional
    """
    if depth_data.empty:
        return
    df = pd.DataFrame(depth_data)

    formatted_symbol = format_symbol(symbol)
    symbol_dir = os.path.join(
        DATA_LOCATION, asset_class, "binance", "minute", formatted_symbol
    )
    os.makedirs(symbol_dir, exist_ok=True)

    # LEAN depth format: ms_midnight, percentage, depth, notional
    df["timestamp"]=pd.to_datetime(df["timestamp"], utc=True)
    df["ms_midnight"] = df["timestamp"].apply(get_ms_from_midnight)

    # Filename: YYYYMMDD_depth.zip
    zip_path = os.path.join(symbol_dir, f"{date_str}_depth.zip")
    zf_name = f"{date_str}_depth.csv"
    
    lean_df = df[["ms_midnight", "percentage", "depth", "notional"]]
    csv_content = lean_df.to_csv(index=False, header=False)

    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
        zf.writestr(zf_name, csv_content)


In [114]:
df=fetch_depth_range_cryptofuture(symbol, since_ms, until_ms, margin_type, data_source)
df

Fetching Depth BTCUSDT: 172800000.0it [00:06, 27202467.04it/s]                               


{'20251231':                  timestamp  percentage     depth      notional
 0      2025-12-31 00:00:09          -5  6703.698  5.804620e+08
 1      2025-12-31 00:00:09          -4  5887.182  5.114127e+08
 2      2025-12-31 00:00:09          -3  4810.885  4.196698e+08
 3      2025-12-31 00:00:09          -2  3724.449  3.258820e+08
 4      2025-12-31 00:00:09          -1  1618.261  1.424802e+08
 ...                    ...         ...       ...           ...
 27185  2025-12-31 23:59:30           1  1173.489  1.031413e+08
 27186  2025-12-31 23:59:30           2  2488.320  2.200352e+08
 27187  2025-12-31 23:59:30           3  3385.077  3.006026e+08
 27188  2025-12-31 23:59:30           4  4036.747  3.596834e+08
 27189  2025-12-31 23:59:30           5  5563.357  4.991079e+08
 
 [27190 rows x 4 columns],
 '20260101':                  timestamp  percentage     depth      notional
 0      2026-01-01 00:00:09          -5  6419.548  5.524643e+08
 1      2026-01-01 00:00:09          -4  5971.786  

In [115]:
for date_str, depth_data in df.items():     
    save_depth_data(symbol, date_str, depth_data)

In [112]:
date_str

'2026-01-01'